kurze Projektbeschreibung


In [1]:
# Install YOLOv8 (Ultralytics)
#!pip install ultralytics       # only for google colab
import cv2
from ultralytics import YOLO
from PIL import Image
import numpy as np
import os


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/tompusch/Library/Mobile Documents/com~apple~CloudDocs/MCI_2/AI project Hollaus/venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, 

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/tompusch/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
video_path = "videos/single_pushup_fabi.MOV"

# Video prüfen
if not os.path.exists(video_path):
    raise FileNotFoundError(f"Video nicht gefunden: {video_path}")

In [4]:
model_pose = YOLO('yolov8s-pose.pt')  # n or s

/Users/tompusch/Library/Mobile Documents/com~apple~CloudDocs/MCI_2/AI project Hollaus/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


100%|██████████| 22.4M/22.4M [00:00<00:00, 30.1MB/s]


In [6]:
import csv

cap = cv2.VideoCapture(video_path)
frame_skip = 10
frame_idx = 0
display_count = 0
max_display = 10

# CSV vorbereiten
csv_file = open("keypoints_output.csv", mode="w", newline="")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["frame", "person_id", "keypoint_id", "x", "y", "confidence"])

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_idx % frame_skip == 0:
        results = model_pose.predict(source=frame, save=False, conf=0.25, verbose=False)

        for i, result in enumerate(results):
            if result.keypoints is not None and result.keypoints.conf is not None:
                keypoints = result.keypoints.xy[0].cpu().numpy()  # (17, 2)
                scores = result.keypoints.conf[0].cpu().numpy()   # (17,)

                for kp_id, ((x, y), conf) in enumerate(zip(keypoints, scores)):
                    csv_writer.writerow([frame_idx, i, kp_id, x, y, conf])

                # Optional anzeigen
                if display_count < max_display:
                    im_array = result.plot()
                    im = Image.fromarray(im_array)
                    display(im)
                    display_count += 1

    frame_idx += 1

cap.release()
csv_file.close()


RuntimeError: Numpy is not available